In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import os
import warnings    
warnings.filterwarnings('ignore')
import glob
import plotly.graph_objects as go
# updated 2021, Nov 11
import sys

In [2]:
from IPython.core.display import display, HTML
display(HTML ('<style>.container {width: 80% !important;}</style>'))

## Plot the s_curve plots

In [3]:
# media driver mapping
wamm_mapping_path = os.path.abspath('../../data/input_data/WMG WAMM Project Data Dictionary_v12.xlsx')

# update response_curve folder for customer and unit sales models. 
# results from new_customer model
customer_folder = '../../models/manual/new_customer/mix_media_metrics/2021-11-05_model_outputs_v2/response_curve_2019-01-01_2021-03-31'
                  
# results from unit_sales model
unit_sales_folder = '../../models/manual/sales_unit/mix_media_metrics/2021-11-09_model_outputs_v1/response_curve_2019-01-01_2021-04-30'
                    
MODEL_DICT = {
    'sales_unit': {'model_graph': 'M1', 
                    'model_results_path': unit_sales_folder},
    'new_customer': {'model_graph': 'M2',
                     'model_results_path': customer_folder}
}

# specify the model type you want to run
# MODEL_TYPE = 'sales_unit'
MODEL_TYPE = 'new_customer'

# title and axis sizes
TITLE_SIZE = 31
AXIS_SIZE = 30

In [4]:
def get_var_description(var_name):
        wamm_mapping = pd.read_excel(wamm_mapping_path, sheet_name = 'Data Dictionary')
        if var_name in wamm_mapping['Variable Name'].values:
            var_filter = (wamm_mapping['Variable Name'] == var_name)
            var_desciption = wamm_mapping[var_filter]['Variable Description'].values[0]
        else:
            var_desciption = var_name
        return var_desciption

In [5]:
four_point_size = 20
other_point_size = 10

files = glob.glob(os.path.join(MODEL_DICT[MODEL_TYPE]['model_results_path'], 'M_*.xlsx'))
for file in files:
    sheet_name = file.rsplit('/')[-1].rsplit('_', 5)[0]
    # 1. edit and format title
    title_desc = get_var_description(sheet_name)    
    if 'TV' in  title_desc:
        title_text = '{} Response Curve'.format(title_desc.strip().title())
        title_text = title_text.replace('Tv', 'TV')
    
    elif 'SBA' in  title_desc:
        title_text = '{} Response Curve'.format(title_desc.strip().title())
        title_text = title_text.replace('Sba', 'SBA')
        title_text = title_text.replace('Sp', 'SP')
        
    elif 'CT' in  title_desc:
        title_text = '{} Response Curve'.format(title_desc.strip().title())
        title_text = title_text.replace('Ct', 'CT')
        title_text = title_text.replace('Ros', 'ROS')

    else:
        title_text = '{} Response Curve'.format(title_desc.strip().title())
        
    if len(title_desc) >= 37:
        width=1250
        height=700 
    
    else:
        width=1150
        height=700

    plot_df = pd.read_excel(file, sheet_name = 's_curve')
    artif_df = pd.read_excel(file, sheet_name = 'Artificial')
    points_df = pd.read_excel(file, sheet_name = 'points')
    
    line1 = go.Scatter(x=plot_df['Raw Variable'], y=plot_df['Contribution'] , mode='markers',
                           marker = dict(color= '#9D9FA2', size=other_point_size), line_shape= 'spline', showlegend=False )

    line6 = go.Scatter(x= artif_df['artificial_x'], y= artif_df['artificial_y'],
                               mode='lines',connectgaps=True, 
                               line=dict(color='#9D9FA2'), line_shape= 'spline', showlegend=False)

    # add the inflection point
    inf_x = points_df[points_df['points'] == 'inflection']['x'].values[0]
    inf_y = points_df[points_df['points'] == 'inflection']['y'].values[0]
    
    # add the threshold point
    thr_x = points_df[points_df['points'] == 'threshold']['x'].values[0]
    thr_y = points_df[points_df['points'] == 'threshold']['y'].values[0]

    # add the saturation point
    sat_x = points_df[points_df['points'] == 'saturation']['x'].values[0]
    sat_y = points_df[points_df['points'] == 'saturation']['y'].values[0]
    
    # add the average point point
    avg_x = points_df[points_df['points'] == 'non_zero_average']['x'].values[0]
    avg_y = points_df[points_df['points'] == 'non_zero_average']['y'].values[0]
    
    four_points = [[inf_x, inf_y, '#FFC220'],[sat_x, sat_y, '#00BEFF'], [thr_x, thr_y, '#FF0000'], [avg_x, avg_y, '#414042']]
    
    data = [line1,  line6]
    for point in four_points:
        line = go.Scatter(x=[point[0]], y=[point[1]], mode='markers+text', connectgaps=True, line=dict(color=point[2]), textfont=dict(family="Bogle/Bogle Regular",size=four_point_size, color=point[2]),marker= dict(size= four_point_size))
        data.append(line)
        
    fig = go.Figure(data = data)
    # update the graphs                                      
    # 2. edit the title,
    fig.update_layout(
    title=title_text,
    xaxis_title='Spends/Day',
    yaxis_title="Volume Contribution",
    font=dict(
        family="Bogle",
        size=TITLE_SIZE,
        color='black', 
        
    ),title_font_family="Bogle", 
        title_x=0.5)
    
    fig.update_xaxes(title_font_size=AXIS_SIZE)
    fig.update_yaxes(title_font_size=AXIS_SIZE)
    
    # 3. remove the x-y axis
    fig.update_yaxes(showticklabels=False, showgrid=False)
    fig.update_xaxes(showticklabels=False, showgrid=False)
    fig.update_layout(showlegend=False)
    
    # 4. update the background color
    fig.update_layout({
    'plot_bgcolor': 'rgba(0, 0, 0, 0)',
    'paper_bgcolor': 'rgba(0, 0, 0, 0)',
    })
    # 5. update the size of the graph
    fig.update_layout(
    autosize=False,
    width=width,
    height=height,)
    fig.show()
    # save the graphs
    # model_type 
    model_type = 'S_Curve'
    graph_folder = os.path.join('./', model_type)
    if not os.path.isdir(graph_folder):
        os.mkdir(graph_folder)
    fig_path = os.path.join(graph_folder, '{model_graph}_{sheet_name}.png'.format(model_graph = MODEL_DICT[MODEL_TYPE]['model_graph'], sheet_name=sheet_name))
    fig.write_image(fig_path) 